In [1]:
import pandas as pd
import numpy as np
import re
import os
import folium

## PD Set Options

In [2]:
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', 800)

##  Read-in data

In [3]:
df = pd.read_csv('redfin_2023-03-07-14-36-18.csv')

## Data validation

In [4]:
### Remove NaNs from 'SOLD DATE' ###
df['SOLD DATE'] = df['SOLD DATE'].fillna('Not provided')

In [5]:
### Filter to just the last month's data ###
x = df['SOLD DATE'].str.startswith('February')
df = df[x]

In [6]:
### Ensure that only 'PROPERTY TYPE' Condo/Co-op is in the data ###
df = df.loc[df['PROPERTY TYPE'] == 'Condo/Co-op']

In [7]:
df.reset_index(inplace=True)
print('Done!')

Done!


## Correction section (if needed)

In [8]:
# df.at[517,'PRICE']=(155000)
# df.at[290,'PRICE']=(550000)

# df.at[517,'$/SQUARE FEET']=(180)
# df.at[290,'$/SQUARE FEET']=(487)

In [9]:
df = df.drop(index=[435])

## Filtering

In [10]:
### Sort properties by sale price, with highest sale price at the top ###
df = df.sort_values(by='PRICE',ascending=False)
### Create a price column formatted as currency ###
df['PRICE_AS_CURRENCY'] = df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
### Set formatting for Beds, Baths ###
df['BEDS'] = df['BEDS'].apply(lambda x: '{:,.0f}'.format(x))
df['BATHS'] = df['BATHS'].apply(lambda x: '{:,.0f}'.format(x))
df['YEAR BUILT'] = df['YEAR BUILT'].apply(lambda x: '{:.0f}'.format(x))
df['PRICE_SQUARE_FEET_AS_CURRENCY'] = df['$/SQUARE FEET'].apply(lambda x: '${:,.0f}'.format(x))

In [11]:
### Insert different colors for top 10 sales vs. the rest ###
df['COLOR'] = ''
### Create RANK column ###
df['RANK'] = 0
### Insert RANK values ###
df['RANK'] = range(1, len(df) + 1)
# use numpy to assign values to the 'COLOR' column
df['COLOR'] = np.where(df['RANK'] <= 10, 'orange', 'blue')

## HTML Popup Formatter

In [12]:
### Define list of columns to drop from DF ###
columns_drop = ['index','SALE TYPE','PROPERTY TYPE','STATE OR PROVINCE','ZIP OR POSTAL CODE','HOA/MONTH','STATUS','NEXT OPEN HOUSE START TIME','NEXT OPEN HOUSE END TIME','URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)','SOURCE','MLS#','FAVORITE','INTERESTED','SQUARE FEET','LOT SIZE']

In [13]:
### Drop the columns ###
df = df.drop(columns=columns_drop)

In [14]:
def popup_html(row):
    Price = row['PRICE_AS_CURRENCY']
    Address = row['ADDRESS']
    City = row['CITY']
    sold_date = row['SOLD DATE']
    beds = row['BEDS']
    baths = row['BATHS']
    psf = row['PRICE_SQUARE_FEET_AS_CURRENCY']
    year_built = row['YEAR BUILT']
    rank = row['RANK']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Price: </strong>{}'''.format(Price) + '''<br>
    <strong>Address: </strong>{}'''.format(Address) + '''<br>
    <strong>City: </strong>{}'''.format(City) + '''<br>
    <strong>Sold: </strong>{}'''.format(sold_date) + '''<br>
    <strong>Beds: </strong>{}'''.format(beds) + '''<br>
    <strong>Baths: </strong>{}'''.format(baths) + '''<br>
    <strong>Price per sf: </strong>{}'''.format(psf) + '''<br>
    <strong>Year Built: </strong>{}'''.format(year_built) + '''<br>
    <strong>Price Rank: </strong>{}'''.format(rank) + '''
    </html>
    '''
    return html

## Make Map

In [15]:
### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=10.5,tiles=None)

### Create title ###
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f"February 2023 Condo Sales")

m.get_root().html.add_child(folium.Element(title_html))

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top 10 Sales')

for index, row in df.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)

# Display map
m

In [16]:
m.save('index.html')

## Summary Info

In [17]:
print('SALES INFO')
print(f'Number of sales: {len(df)}')
print('--------')
print(f'Total sale price: ${df["PRICE"].sum():,.0f}')
print('--------')
print(f'Median sale price: ${df["PRICE"].median():,.0f}')
print('--------')
print(f'Max sale price: ${df["PRICE"].max():,.0f}')
print('--------')
print(f'Min sale price: ${df["PRICE"].min():,.0f}')
print('------------------------------------------------')
print('PSF INFO')
print(f'Max price per square foot: ${df["$/SQUARE FEET"].max():,.0f}')
print('--------')
print(f'Min price per square foot: ${df["$/SQUARE FEET"].min():,.0f}')
print('--------')
print(f'Median price per square foot: ${df["$/SQUARE FEET"].median():,.0f}')
print('------------------------------------------------')
print('CONDO AGES')
print(f'Newest building: {df["YEAR BUILT"].max()}')
print('----------')
print(f'Oldest building: {df["YEAR BUILT"].min()}')
print('----------')
print(f'Average building age: {df["YEAR BUILT"].mean()}')
print('------------------------------------------------')
print('BEDS & BATHS')
print(f'Most beds: {df["BEDS"].max()}')
print('----------')
print(f'Fewest beds: {df["BEDS"].min()}')
print('----------')
print(f'Most baths: {df["BATHS"].max()}')
print('----------')
print(f'Fewest baths: {df["BATHS"].min()}')

SALES INFO
Number of sales: 581
--------
Total sale price: $255,496,095
--------
Median sale price: $249,900
--------
Max sale price: $7,400,000
--------
Min sale price: $60,000
------------------------------------------------
PSF INFO
Max price per square foot: $3,645
--------
Min price per square foot: $43
--------
Median price per square foot: $229
------------------------------------------------
CONDO AGES
Newest building: 2021
----------
Oldest building: 1948
----------
Average building age: inf
------------------------------------------------
BEDS & BATHS
Most beds: 4
----------
Fewest beds: 0
----------
Most baths: 4
----------
Fewest baths: 1


In [18]:
df.columns

Index(['SOLD DATE', 'ADDRESS', 'CITY', 'PRICE', 'BEDS', 'BATHS', 'LOCATION',
       'YEAR BUILT', 'DAYS ON MARKET', '$/SQUARE FEET', 'LATITUDE',
       'LONGITUDE', 'PRICE_AS_CURRENCY', 'PRICE_SQUARE_FEET_AS_CURRENCY',
       'COLOR', 'RANK'],
      dtype='object')

In [19]:
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

Newest = '\033[1m' + 'Newest' + '\033[0m'
Oldest = '\033[1m' + 'Oldest' + '\033[0m'

In [20]:
### Convert 'YEAR BUILT' back to integer ###
df['YEAR BUILT'] = pd.to_numeric(df['YEAR BUILT'])

In [21]:
print(f"{ME}{BR}{df.loc[df['PRICE'].idxmax()]['LOCATION']}, {df.loc[df['PRICE'].idxmax()]['ADDRESS']} | Price ${df.loc[df['PRICE'].idxmax()]['PRICE']:,.0f} | ${df.loc[df['PRICE'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df.loc[df['PRICE'].idxmax()]['YEAR BUILT']}")
print(f"{LE}{BR}{df.loc[df['PRICE'].idxmin()]['LOCATION']}, {df.loc[df['PRICE'].idxmin()]['ADDRESS']} | Price ${df.loc[df['PRICE'].idxmin()]['PRICE']:,.0f} | ${df.loc[df['PRICE'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df.loc[df['PRICE'].idxmin()]['YEAR BUILT']}")

print(f"{MAX_PSF}{BR}{df.loc[df['$/SQUARE FEET'].idxmax()]['LOCATION']}, {df.loc[df['$/SQUARE FEET'].idxmax()]['ADDRESS']} | Price ${df.loc[df['$/SQUARE FEET'].idxmax()]['PRICE']:,.0f} | ${df.loc[df['$/SQUARE FEET'].idxmax()]['$/SQUARE FEET']:,.0f} psf | Year built: {df.loc[df['$/SQUARE FEET'].idxmax()]['YEAR BUILT']}")
print(f"{MIN_PSF}{BR}{df.loc[df['$/SQUARE FEET'].idxmin()]['LOCATION']}, {df.loc[df['$/SQUARE FEET'].idxmin()]['ADDRESS']} | Price ${df.loc[df['$/SQUARE FEET'].idxmin()]['PRICE']:,.0f} | ${df.loc[df['$/SQUARE FEET'].idxmin()]['$/SQUARE FEET']:,.0f} psf | Year built: {df.loc[df['$/SQUARE FEET'].idxmin()]['YEAR BUILT']}")

print(f"{Newest}{BR}{df.loc[df['YEAR BUILT'].idxmax()]['LOCATION']}, {df.loc[df['YEAR BUILT'].idxmax()]['ADDRESS']} | Price ${df.loc[df['YEAR BUILT'].idxmax()]['PRICE']:,.0f} | ${df.loc[df['YEAR BUILT'].idxmax()]['$/SQUARE FEET']} psf | Year built: {df.loc[df['YEAR BUILT'].idxmax()]['YEAR BUILT']}")
print(f"{Oldest}{BR}{df.loc[df['YEAR BUILT'].idxmin()]['LOCATION']}, {df.loc[df['YEAR BUILT'].idxmin()]['ADDRESS']} | Price ${df.loc[df['YEAR BUILT'].idxmin()]['PRICE']:,.0f} | ${df.loc[df['YEAR BUILT'].idxmin()]['$/SQUARE FEET']} psf | Year built: {df.loc[df['YEAR BUILT'].idxmin()]['YEAR BUILT']}")

Most Expensive
Sun And Surf One Hundred And Thirty Condos, 100 Sunrise Ave #611 | Price $7,400,000 | $3,645 psf | Year built: 1977
Least Expensive
Southport AT Hunters Run Condo, 37 Southport Ln Unit F | Price $60,000 | $43 psf | Year built: 1980
Highest Price Per Square Foot
Sun And Surf One Hundred And Thirty Condos, 100 Sunrise Ave #611 | Price $7,400,000 | $3,645 psf | Year built: 1977
Lowest Price Per Square Foot
Southport AT Hunters Run Condo, 37 Southport Ln Unit F | Price $60,000 | $43 psf | Year built: 1980
Newest
Ocean Delray Condo, 1901 S Ocean Blvd #2 | Price $5,600,000 | $1566.0 psf | Year built: 2021
Oldest
Gulfstream Condo, 31 S Golfview Rd #8 | Price $115,000 | $307.0 psf | Year built: 1948


In [27]:
from datetime import datetime, timedelta

date1 = datetime(2022, 12, 27) ## List (Earlier) date
date2 = datetime(2023, 2, 10) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

45


## Map URL Snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [23]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

In [24]:
cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/February_2023_PalmBeach_monthly_condo_sales
